In [ ]:
import os
import snowflake.connector as sf_c
import boto3
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

s3_resource = boto3.resource('s3')
s3_client = boto3.client('s3')

CEDL_HOME = os.environ['CEDL_HOME']
nexus_connectionProfile = CEDL_HOME + '/etc/.sf.nexus.profile'
s3_connectionProfile = CEDL_HOME + '/etc/.s3_connection_profile'



def snowFlake_Connection():
    try:
        pathExist = os.path.exists(nexus_connectionProfile)
        if (not pathExist):
            print('The profile {} doesn''t exist'.format(nexus_connectionProfile))
            exit(1)
        profileFile = open(nexus_connectionProfile)
        for line in profileFile:
            if (line.split('=')[0] == 'snowflakeAccount'):
                snowflakeAccount = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeUsername'):
                snowflakeUsername = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakePassword'):
                snowflakePassword = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeRole'):
                snowflakeRole = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeDBName'):
                snowflakeDBName = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeWarehouse'):
                snowflakeWarehouse = line.split('=')[1].replace('\n', '')
            else:
                pass
        profileFile.close()
        if (len(snowflakeAccount) == 0 or len(snowflakeUsername) == 0 or len(snowflakePassword) == 0 or len(
                snowflakeRole) == 0 or len(snowflakeDBName) == 0 or len(snowflakeWarehouse) == 0):
            print('some parameters are missing from {}'.format(nexus_connectionProfile))
            exit(1)
        conn = sf_c.connect(user=snowflakeUsername, password=snowflakePassword, account=snowflakeAccount,
                            warehouse=snowflakeWarehouse, database=snowflakeDBName)
        print("connected to SNOWFLAKE Database.")
    except sf_c.Error as e:
        print('Error connecting to SNOWFLAKE Database - {}'.format(e))
        exit(1)
    return conn

def load_csv_files_into_snowflake(folder_path, snowflake_config):

    try:
        # List all files in the specified folder
        files = os.listdir(folder_path)

        # Create a connection to Snowflake
        conn = snowFlake_Connection()
        cursor = conn.cursor()

        # Iterate through each file in the folder
        for file in files:
            # Check if the file is a CSV file (you can modify this condition based on your file types)
            if file.endswith(".csv"):
                # Generate a unique table name for each file
                table_name = file.split(".")[0]  # Use the file name as the table name

                # Upload the CSV file to Snowflake using the COPY command
                upload_csv_to_snowflake(conn, folder_path, file, table_name, snowflake_config.get('schema'), snowflake_config.get('database'))

    except snowflake.connector.errors.DatabaseError as e:
        print(f"Snowflake Database Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the Snowflake connection in the finally block to ensure it's always closed
        if conn:
            conn.close()

def upload_csv_to_snowflake(connection, folder_path, file_name, table_name, schema, database):
    try:
        # Create a Snowflake cursor
        cursor = connection.cursor()

        # Use the COPY INTO command to create and load data into the Snowflake table
        copy_command = f"COPY INTO {database}.{schema}.{table_name} FROM @%/{file_name} FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY = '\"')"
        cursor.execute(copy_command)

        # Commit the changes
        connection.commit()
    except snowflake.connector.errors.DatabaseError as e:
        print(f"Snowflake Database Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor in the finally block to ensure it's always closed
        if cursor:
            cursor.close()

# Example usage:
folder_path = "/AppDev/CEDL/etl/SrcFiles/lg/foo"

snowflake_config = {
    'database': 'NEXUS',
    'schema': 'TEMP'
}

load_csv_files_into_snowflake(folder_path, snowflake_config)
